In [2]:
import pandas as pd
from utilities_DL import get_DataSet_and_invalid_dates,get_MultiModel_loss_args_emb_opts,load_init_trainer,load_prediction
from DL_class import MultiModelTrainer, Trainer
from config import get_args
from save_results import build_results_df
from paths import folder_path,file_name,get_save_directory
import time 
import torch
import argparse

from ray_config import get_ray_config
import ray 
from ray import tune 

# ==== GET PARAMETERS ====
# Load config
model_name = 'STGCN'  #'CNN'
args = get_args(model_name)
#args = get_args(model_name = model_name,learn_graph_structure = True)  # MTGNN

# Modification :
args.epochs = 350
args.K_fold = 6   # Means we will use the first fold for the Ray Tuning and the 5 other ones to get the metrics
if torch.cuda.is_available():
    args.device = 'cuda:0'
    args.batch_size = 256
else :
    args.device = 'cpu'
    args.batch_size = 32

args.single_station = True
args.ray = True

args.loss_function_type =   'quantile' #'MSE' 

if args.loss_function_type == 'MSE':
    args.out_dim = 1
    args.alpha = None
    args.track_pi = False

else:
    args.out_dim = 2
    args.alpha = 0.1
    args.track_pi = True


print("!!! Prediction sur une UNIQUE STATION et non pas les 40 ") if args.single_station else None
print(f"!!! Loss function: {args.loss_function_type} ") 
print(f"Model: {args.model_name}, K_fold = {args.K_fold}") 
    

!!! Prediction sur une UNIQUE STATION et non pas les 40 
!!! Loss function quantile 
Model: STGCN, K_fold = 6


In [3]:
# Validation
# Quantile 
print(100*(1-0.081604/0.083117),100*(1-0.082376/0.083117))

# MSE
print(100*(1-0.001440/0.001475), 100*(1-0.001508/0.001475))

1.8203255651671801 0.8915143713078999
2.3728813559321993 -2.2372881355932295


In [4]:
#Test
# Quantile
print(100*(1-0.082328/0.083057),100*(1-0.083235/0.083057))

# MSE
print(100*(1-0.001584/0.001878), 100*(1-0.001717/0.001878))

0.877710487978145 -0.21431065412909067
15.65495207667732 8.572949946751862


## Search Space : 

In [3]:
config = {"lr": tune.qloguniform(1e-4, 1e-1, 5e-5),
          "weight_decay" : tune.uniform(0.0005, 0.1),
          "momentum" : tune.uniform(0.85, 0.99),
          "dropout" : tune.uniform(0,0.9),
        }

config_embedding = {#'calendar_class' : tune.choice([1,2,3]),
                    'embedding_dim' : tune.choice([2,3,4,5,6]),
                    'multi_embedding' : tune.choice([True,False]),
                    #'TE_transfer' : tune.choice([True,False]),
                    }


config_stgcn = {"Kt" : tune.choice([2,3,4]),
                "stblock_num" : tune.choice([1,2,3,4]),
                "act_fun" : tune.choice(['glu','gtu']),
                "Ks" :  tune.choice([2,3]),
                "graph_conv_type" : tune.choice(['cheb_graph_conv','graph_conv']),
                "gso_type" : tune.choice(['sym_norm_lap', 'rw_norm_lap', 'sym_renorm_adj', 'rw_renorm_adj']),
                "adj_type" : 'dist',
                }

if args.time_embedding:
    config.update(config_embedding)

if args.model_name == 'STGCN':
     config.update(config_stgcn)

## Hyper Parameter Tuning sur le Fold 0

In [4]:
def load_trainer(config,folder_path,file_name,args):

    for key, value in config.items():
        if hasattr(args, key):
            setattr(args, key, value)

    Datasets,DataLoader_list,dic_class2rpz,nb_words_embedding,time_slots_labels,dic_rpz2class = load_init_trainer(folder_path,file_name,args)
    (loss_function,Model_list,Optimizer_list,Scheduler_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))
    dataset,dataloader,model,optimizer,scheduler = Datasets[0],DataLoader_list[0],Model_list[0],Optimizer_list[0],Scheduler_list[0]


    trainer = Trainer(dataset,model,dataloader,
                    args,optimizer,loss_function,scheduler = scheduler,
                    args_embedding=args_embedding,
                    save_dir = None,dic_class2rpz=dic_class2rpz)
    return(trainer)

def Train_with_tune(config):
    trainer = load_trainer(config,folder_path,file_name,args)
    result_df = trainer.train_and_valid()



ray_scheduler,ray_search_alg,resources_per_trial,num_gpus,max_concurrent_trials,num_cpus = get_ray_config(args)


if ray.is_initialized:
    ray.shutdown()
    ray.init(num_gpus=num_gpus,num_cpus=num_cpus)

config = {"lr": tune.choice([1e-4])}
analysis = tune.run(
        Train_with_tune,
        config=config,
        num_samples=1,  # Increase num_samples for more random combinations
        resources_per_trial = resources_per_trial,
        max_concurrent_trials = max_concurrent_trials,
        scheduler = ray_scheduler,
        search_alg = ray_search_alg,
    )

ValueError: not enough values to unpack (expected 6, got 5)

In [5]:
def load_trainer(folder_path,file_name,args):
    Datasets,DataLoader_list,dic_class2rpz,nb_words_embedding,time_slots_labels,dic_rpz2class = load_init_trainer(folder_path,file_name,args)
    (loss_function,Model_list,Optimizer_list,Scheduler_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))
    dataset,dataloader,model,optimizer,scheduler = Datasets[0],DataLoader_list[0],Model_list[0],Optimizer_list[0],Scheduler_list[0]


    trainer = Trainer(dataset,model,dataloader,
                    args,optimizer,loss_function,scheduler = scheduler,
                    args_embedding=args_embedding,
                    save_dir = None,dic_class2rpz=dic_class2rpz)
    return(trainer)

def Train_with_torch_only(folder_path,file_name,args):
    args.ray = False
    trainer = load_trainer(folder_path,file_name,args)
    result_df = trainer.train_and_valid()
    return(result_df, trainer)

result_df, trainer = Train_with_torch_only(folder_path,file_name,args)

coverage period: 2019-01-01 00:00:00 - 2020-01-01 00:00:00
Time-step per hour: 4.0
start training


2024-05-17 10:46:11,601	WARNING session.py:91 -- Session not detected. You should not be calling `report` outside `tuner.fit()` or while using the class API. 
2024-05-17 10:46:11,601	WARNING session.py:97 --   File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/Users/romainrochas/opt/anaconda3/envs

In [18]:
from plotting_bokeh import plot_loss,plot_prediction
from bokeh.plotting import output_notebook,show
p = plot_loss(trainer,location = "top_right")
output_notebook()
show(p)

Loading BokehJS ...

In [14]:
Datasets,DataLoader_list,dic_class2rpz,nb_words_embedding,time_slots_labels,dic_rpz2class = load_init_trainer(folder_path,file_name,args)
(loss_function,Model_list,Optimizer_list,Scheduler_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))
dataset,dataloader,model,optimizer,scheduler = Datasets[0],DataLoader_list[0],Model_list[0],Optimizer_list[0],Scheduler_list[0]

coverage period: 2019-01-01 00:00:00 - 2020-01-01 00:00:00
Time-step per hour: 4.0


In [23]:
Q = trainer.conformal_calibration(args.alpha,dataset,conformity_scores_type = args.conformity_scores_type,quantile_method = args.quantile_method)

Proportion of label with quantile order set to 1: 23.8%


In [17]:
preds,Y,T = load_prediction(trainer,dataset,dataloader,args,training_mode='test',normalize=False)

In [26]:
(pi,pi_cqr,p) = plot_prediction(trainer,dataset,Q,args,station = 0, location = "top_right")
output_notebook()
show(p)

Loading BokehJS ...

In [9]:
from torch.optim.lr_scheduler import ConstantLR,ExponentialLR,SequentialLR,LinearLR
from torch.optim import AdamW
from dl_models.CNN_based_model import CNN
from config import get_args
import pandas as pd
import numpy as np 
model_name = 'CNN'
args = get_args(model_name)

args.torch_scheduler_milestone = 50
args.torch_scheduler_lr_start_factor = 0.01
args.torch_scheduler_gamma = 0.999

model = CNN(args, kernel_size = (2,),args_embedding = None,dic_class2rpz = None)

optimizer = AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

#scheduler1 = ConstantLR(optimizer, factor=0.1, total_iters=2)
scheduler1 = LinearLR(optimizer,total_iters = args.torch_scheduler_milestone, start_factor = args.torch_scheduler_lr_start_factor)
scheduler2 = ExponentialLR(optimizer, gamma=args.torch_scheduler_gamma)
scheduler = SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[args.torch_scheduler_milestone])
milestone = 15
print(args.lr)



lr_list = []
for epoch in range(500):
    scheduler.step()
    current_lr = scheduler.optimizer.param_groups[0]['lr']
    lr_list.append(current_lr)

import bokeh
from bokeh.plotting import figure, show

p = figure()
p.line(np.arange(len(lr_list)),lr_list)
show(p)

0.0001


In [10]:
1/16

'cuda:1'